In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import time
import ollama
import json
from utils import load_json, save_json

In [49]:
cases = {'1':[], '2':[], '3':[], '4':[]}
for root, dirs, files in os.walk('../pilot_study_sample/'):
    for dir in dirs:
        for root, dirs, files in os.walk('../pilot_study_sample/' + dir):
            for file in files:
                if file.endswith('.json'):
                    cases[dir].append(os.path.join(root, file))

In [51]:
from summary_agent import SummaryAgent

summary_agent = SummaryAgent(backend="groq")
i4_case = load_json(cases['4'][0])
summary_i4case = summary_agent.summarize_case(i4_case)

Token indices sequence length is longer than the specified maximum sequence length for this model (4120 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (3928 > 512). Running this sequence through the model will result in indexing errors


In [79]:
importance_id_to_level = {'1':'High', '2':'High', '3':'Medium', '4':'Low'}

In [80]:
summary_i4case['importance'] = importance_id_to_level[summary_i4case['importance']]

In [82]:
import re

def get_last_question(text):
    # Regex pattern to match questions, considering multiline questions
    pattern = r'Question:\s*(.*?)(?=\n[A-Z]|\Z)'

    # Find all matches with the multiline flag enabled
    matches = re.findall(pattern, text, re.DOTALL)
    
    # Get the last match
    last_question = matches[-1] if matches else None

    return last_question


def check_for_stop(text):
    # Regex pattern to match questions, considering multiline questions
    pattern = r'Stop:\s*(.*?)(?=\n[A-Z]|\Z)'

    # Find all matches with the multiline flag enabled
    matches = re.findall(pattern, text, re.DOTALL)
    
    # Get the last match
    stop = matches[-1] if matches else None

    return stop


In [100]:
from senior_agent import SeniorAgent
from junior_agent import JuniorAgent

In [101]:
s_agent = SeniorAgent()
j_agent = JuniorAgent()

In [102]:
round = 20

for i in range(round):
    print(f"---------------------round{i+1}-------------------------")
    response = s_agent.ask_question()
    question = get_last_question(response)
    stop = check_for_stop(response)
    if (stop is not None) or (question is None):
        print(f"Stop: {stop}")
        print(response)
        break

    answer = j_agent.answer_question(summary_i4case, question)
    s_agent.append_to_cache(answer)
    
    print(f"Senior Agent: {question}")
    print(f"Junior Agent: {answer}")
    time.sleep(20)

---------------------round1-------------------------
Senior Agent: Can I have a brief summary of the facts of the case, including the key issues and complaints brought before the Court by the applicant(s)?
Junior Agent: The applicant, Vyacheslav Aleksandrovich Gordiyenko, a Ukrainian national, alleged that he was subjected to ill-treatment by law enforcement officials on several occasions, including in 2007 and 2008. He claimed that the authorities failed to conduct an effective investigation into these incidents, despite his complaints and requests for protection. Specifically, the applicant alleged that he was attacked by T. at a trolleybus station in 2006 and by an unknown assailant in 2008, and was involved in an incident at the District Court in 2010, where he was allegedly ill-treated by court security guards and police officers. The applicant complained to the police about these incidents, but the investigations were repeatedly closed due to lack of evidence or lack of corpus de

In [103]:
print(response)

Final Answer:

Based on the analysis of the case, I would conclude that the Court assigned a Low importance score to this case. The basis for my classification is that the case primarily applied established case law and principles regarding Article 3 of the Convention, without breaking new ground in terms of the interpretation or application of the Convention. The Court's judgment found violations of Article 3, but the awards for non-pecuniary damage were relatively low, and the case does not involve any novel or significant legal issues or developments in the interpretation of the Convention. Therefore, the case is classified as having little legal interest, as it simply applies existing case-law, and does not make a significant contribution to the development, clarification, or modification of the Court's case law.


In [125]:
cases.keys()

dict_keys(['1', '2', '3', '4'])

In [122]:
def run_conversation(case_summary):
        s_agent = SeniorAgent()
        j_agent = JuniorAgent()
        round = 20
        scratch_pad = ""
        for i in range(round):
            scratch_pad += f"---------------------round{i+1}-------------------------"+'\n'
            print(f"---------------------round{i+1}-------------------------")
            response = s_agent.ask_question()
            question = get_last_question(response)
            stop = check_for_stop(response)
            if (stop is not None) or (question is None):
                print(f"Stop: {stop}")
                scratch_pad += f"Senior Agent: {response}"+'\n'
                print(response)
                break

            answer = j_agent.answer_question(case_summary, question)
            s_agent.append_to_cache(answer)
            
            print(f"Senior Agent: {question}")
            print(f"Junior Agent: {answer}")

            scratch_pad += f"Senior Agent: {question}"+'\n'
            scratch_pad += f"Junior Agent: {answer}"+'\n'
            time.sleep(20)
        return scratch_pad

In [132]:
not_yet_cases = [
  #'../pilot_study_sample/2/001-57416.json',
  '../pilot_study_sample/3/001-67200.json',
  '../pilot_study_sample/3/001-205675.json',
  '../pilot_study_sample/3/001-57899.json',
  '../pilot_study_sample/3/001-57885.json',
  '../pilot_study_sample/4/001-147041.json'
]

In [133]:
for key  in cases.keys():
    for idx, case_path in enumerate(cases[key]):
        if case_path not in not_yet_cases:
            continue
        print(f"case: {case_path}, {idx+1}/5 ...")
        case = load_json(case_path)
        summary_case = summary_agent.summarize_case(case)
        scratch_pad = run_conversation(summary_case)
        output_dict = {'actual_importance':case['importance'], 'importance_reasoning':scratch_pad}
        save_json('./output/'+case_path.split('/')[-1].replace('.json', '_output.json'), output_dict)
        print(f"    output saved, I will sleep now for 1 minute.")
        # sleep for 1 second to avoid rate limit
        time.sleep(60)


case: ../pilot_study_sample/3/001-67200.json, 1/5 ...


Token indices sequence length is longer than the specified maximum sequence length for this model (1537 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (2369 > 512). Running this sequence through the model will result in indexing errors


---------------------round1-------------------------
Senior Agent: Can my assistant provide me with the case details, including the facts, procedure, and legal arguments presented by the parties involved? I'd like to understand the context of the case before diving deeper.
Junior Agent: Answer:

The case details, including the facts, procedure, and legal arguments presented by the parties involved, are as follows:

**Facts:**

The case concerns three British Army soldiers, Mr. Miller, Mr. Morrison, and Mr. Gillespie, who were charged with various crimes and convicted by court-martial. They appealed their convictions to the Court Martial Appeal Court (CMAC), arguing that they did not have a fair trial by an independent and impartial tribunal within the meaning of Article 6(1) of the European Convention on Human Rights (ECHR).

**Procedure:**

The applicants lodged three applications (nos. 45825/99, 45826/99, and 45827/99) with the European Court of Human Rights (ECHR) under Article 34 o

Token indices sequence length is longer than the specified maximum sequence length for this model (896 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (8549 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (8549 > 512). Running this sequence through the model will result in indexing errors


text has 6734 words and 236 sentences, on average 28.533898305084747 word/sentecesand 8549 tokens, token limit is 7500


Token indices sequence length is longer than the specified maximum sequence length for this model (1209 > 512). Running this sequence through the model will result in indexing errors


---------------------round1-------------------------


RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama3-70b-8192` in organization `org_01hyzgaf4ten29dwzg398849s2` on tokens per minute (TPM): Limit 6000, Used 0, Requested 8029. Please try again in 20.29s. Visit https://console.groq.com/docs/rate-limits for more information.', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}